In [1]:
!pip install rank-bm25 faiss-cpu qdrant_client

In [188]:
# библиотеки и инструменты использовавшиеся в процессе экспериментов (для текущей версии актуальны не все)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd
import re
from sentence_transformers import SentenceTransformer, util
import faiss
from rank_bm25 import BM25Okapi
import time
from tqdm import tqdm
from qdrant_client.models import VectorParams, Distance, PointStruct
from nltk.tokenize import word_tokenize
from multiprocessing import Pool, cpu_count
from sklearn.preprocessing import normalize
import faiss
from scipy.spatial import distance
from fastembed import SparseTextEmbedding, TextEmbedding, LateInteractionTextEmbedding
from concurrent.futures import ThreadPoolExecutor
from qdrant_client import QdrantClient, models
from transformers import AutoTokenizer, AutoModel

# Очистка данных

In [3]:
data_path = "https://docs.google.com/uc?export=download&id=1dgnwG1AZhspMJaDR6jz828qZkSnZg25t"
df_base = pd.read_csv(data_path)
df_base

,context,question,answer
0,Caption: Tasmanian berry grower Nic Hansen sho...,What is the Berry Export Summary 2028 and what...,The Berry Export Summary 2028 is a dedicated e...
1,RWSN Collaborations\nSouthern Africa Self-supp...,What are some of the benefits reported from ha...,Benefits reported from having access to Self-s...
2,All Android applications categories\nDescripti...,What are the unique features of the Coolands f...,The unique features of the Coolands for Twitte...
3,"How unequal is India? The question is simple, ...",What is the main difference between the Nation...,The main difference between the NSS and the IH...
4,Gunnar Nelson took his time on the feet agains...,How did Gunnar Nelson win the fight against Za...,Gunnar Nelson won the fight against Zak Cummin...
...,...,...,...
11995,"Fuzzy's Ultra Premium Vodka\nThe Myth, The Man...",What are some of the achievements of Fuzzy Zoe...,Fuzzy Zoeller is known for his golfing success...
11996,Swedish Grand Prix rider Malin Nilsson got mar...,Who did Malin Nilsson marry on 2 June 2018?,"Malin Nilsson got married to her partner, Germ..."
11997,The Cracchiolo Law Library of the James E. Rog...,What is the Fellowship in Law Librarianship of...,The Fellowship in Law Librarianship is a progr...
11998,2nd physical eMAG store opens in Mammut\nOnlin...,Where has the second physical eMAG store been ...,The second physical eMAG store has been opened...


In [4]:
df = df_base.copy()
df.isna().sum()

context     0
question    3
answer      3
dtype: int64

In [5]:
df = df.dropna()
def clean_text(text):
    if isinstance(text, str):
        text = str(text)
        text = text.lower()  # Приводим текст к нижнему регистру
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Удаляем всё, кроме букв и пробелов
        text = re.sub(r'\s+', ' ', text)  # Заменяем множественные пробелы одним
        return text.strip()  # Удаляем лишние пробелы в начале и конце
    return text
df['context'] = df['context'].apply(clean_text)
df['question'] = df['question'].apply(clean_text)
df['answer'] = df['answer'].apply(clean_text)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11997 entries, 0 to 11999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   context   11997 non-null  object
 1   question  11997 non-null  object
 2   answer    11997 non-null  object
dtypes: object(3)
memory usage: 374.9+ KB


In [7]:
df.isna().sum()

context     0
question    0
answer      0
dtype: int64

In [8]:
df.values[0]

array(['caption tasmanian berry grower nic hansen showing macau chef antimo merone around his property as part of export engagement activities the rise and rise of the australian strawberry raspberry and blackberry industries has seen the sectors redouble their international trade focus with the release of a dedicated export plan to grow their global presence over the next 10 years driven by significant grower input the berry export summary 2028 maps the sectors current position where they want to be highopportunity markets and next steps hort innovation trade manager jenny van de meeberg said the value and volume of raspberry and blackberry exports rose by 100 per cent between 2016 and 2017 she said the australian strawberry industry experienced similar success with an almost 30 per cent rise in export volume and a 26 per cent rise in value to 326m over the same period australian berry sectors are in a firm position at the moment she said production adoption of protected substrate cro

In [9]:
df

,context,question,answer
0,caption tasmanian berry grower nic hansen show...,what is the berry export summary 2028 and what...,the berry export summary 2028 is a dedicated e...
1,rwsn collaborations southern africa selfsupply...,what are some of the benefits reported from ha...,benefits reported from having access to selfsu...
2,all android applications categories descriptio...,what are the unique features of the coolands f...,the unique features of the coolands for twitte...
3,how unequal is india the question is simple th...,what is the main difference between the nation...,the main difference between the nss and the ih...
4,gunnar nelson took his time on the feet agains...,how did gunnar nelson win the fight against za...,gunnar nelson won the fight against zak cummin...
...,...,...,...
11995,fuzzys ultra premium vodka the myth the man th...,what are some of the achievements of fuzzy zoe...,fuzzy zoeller is known for his golfing success...
11996,swedish grand prix rider malin nilsson got mar...,who did malin nilsson marry on 2 june 2018,malin nilsson got married to her partner germa...
11997,the cracchiolo law library of the james e roge...,what is the fellowship in law librarianship of...,the fellowship in law librarianship is a progr...
11998,2nd physical emag store opens in mammut online...,where has the second physical emag store been ...,the second physical emag store has been opened...


In [10]:
print('Number of duplicates context:', df[df['context'].duplicated(keep=False)].shape[0])
print('Number of duplicates question:', df[df['question'].duplicated(keep=False)].shape[0])
print('Number of duplicates answer:', df[df['answer'].duplicated(keep=False)].shape[0])

Количество дубликатов context: 0
Количество дубликатов question: 22
Количество дубликатов answer: 0


In [11]:
df[df['question'].duplicated(keep=False)].sort_values(by='question')

,context,question,answer
9991,background nectar mattress review is this foam...,what are some of the features that set the nec...,some of the features that set the nectar mattr...
6225,background nectar mattress review is this foam...,what are some of the features that set the nec...,the nectar mattress is a reasonably priced foa...
9259,explore concrete pump operator careers and fin...,what are some of the job roles mentioned in th...,some of the job roles mentioned in the context...
9070,lead teacher jobs teacher early childhoodnew p...,what are some of the job roles mentioned in th...,some of the job roles mentioned in the context...
11587,im sorry the pictures are so darksomeone im no...,what are the ingredients needed for the recipe...,the ingredients needed for the recipe are 5 c ...
2599,description if i eat flour i prefer to eat rye...,what are the ingredients needed for the recipe...,the ingredients needed for the recipe are rye ...
10753,hose hook up for jet skichameleon dating sites...,what is the context about,the context is about various topics including ...
8322,they are both comparatively late age on na rie...,what is the context about,the context is a collection of various sentenc...
11483,order by popularity order by alphabet like spr...,what is the context about,the context is about a list of titles or phras...
6217,the synthetic fragrant compounds based on 2ter...,what is the context about,the context is about synthetic fragrant compou...


In [12]:
df_b = df.drop_duplicates(subset=['question'], keep='first')
df_b.reset_index(drop=True, inplace=True)
print('Number of duplicates question:', df_b[df_b['question'].duplicated(keep=False)].shape[0])

Количество дубликатов question: 0


При очистке данных удалены все знаки не несущие полезной информации, удалены все явные дубликаты, данные приведены к нижнему регистру

# Testing Models v.5

In [208]:
#BM25
contexts = [word_tokenize(context) for context in df_b['context']]
questions = [word_tokenize(question) for question in df_b['question']]
answers = df_b['answer'].tolist()
# Инициализация BM25
bm25 = BM25Okapi(contexts)
# Проверка точности на датасете
def evaluate_bm25(df_b, bm25):
    correct = 0
    for i in tqdm(range(len(df_b)), unit="question"):
        question = df_b.iloc[i]['question']
        true_answer = df_b.iloc[i]['answer']
        tokenized_question = word_tokenize(question)
        scores = bm25.get_scores(tokenized_question)
        top_index = scores.argmax()
        retrieved_answer = df_b.iloc[top_index]['answer']
        if retrieved_answer == true_answer:
            correct += 1
    accuracy = correct / len(df_b)
    return accuracy
# Вычисление точности
accuracy = evaluate_bm25(df_b, bm25)
print(f'BM25Okapi accuracy: {accuracy * 100:.2f}%')

100%|██████████████████████████████████████████████████████████████████████| 11983/11983 [25:40<00:00,  7.78question/s]

BM25Okapi accuracy: 87.31%


In [ ]:
#BM25 with ANN
contexts = [word_tokenize(context) for context in df_b['context']]
questions = df_b['question'].tolist()
answers = df_b['answer'].tolist()
# Инициализация BM25
bm25 = BM25Okapi(contexts)
# Параллельное вычисление BM25-оценок
def compute_bm25_parallel(tokenized_documents, bm25):
    with Pool(cpu_count()) as pool:
        scores = list(tqdm(pool.imap(bm25.get_scores, tokenized_documents), total=len(tokenized_documents), unit="context"))
    return scores
# Создание матрицы BM25 для контекстов
bm25_matrix = compute_bm25_parallel(contexts, bm25)
bm25_matrix = np.array(bm25_matrix)
# Обучение ANN модели на основе BM25
ann_bm25_model = NearestNeighbors(n_neighbors=1, metric='cosine').fit(bm25_matrix)

In [ ]:
questions = [word_tokenize(question) for question in df_b['question']]
question_matrix = compute_bm25_parallel(questions, bm25)
question_matrix = np.array(question_matrix)
question_matrix = normalize(question_matrix, norm='l2')

In [ ]:
# Проверка точности
def evaluate_ann_bm25(df_b, question_matrix, ann_bm25_model):
    correct = 0
    for i in tqdm(range(len(df_b)), unit="question"):
        true_answer = df_b.iloc[i]['answer']
        question_scores = question_matrix[i].reshape(1, -1)
        distances, indices = ann_bm25_model.kneighbors(question_scores)
        top_index = indices[0][0]
        retrieved_answer = df_b.iloc[top_index]['answer']
        if retrieved_answer == true_answer:
            correct += 1
    accuracy = correct / len(df_b)
    return accuracy
# Вычисление точности ANN с BM25
accuracy_ann_bm25 = evaluate_ann_bm25(df_b, question_matrix, ann_bm25_model)
print(f'ANN with BM25 Top-1 Accuracy: {accuracy_ann_bm25 * 100:.2f}%')

Evaluating ANN with BM25: 100%|██████████████████████████████████████████| 11983/11983 [3:03:20<00:00,  1.09question/s]

ANN with BM25 Top-1 Accuracy: 8.55%


In [103]:
# Sentence Transformer 'all-MiniLM-L6-v2'
# Подготовка данных
contexts = df_b['context'].tolist()
questions = df_b['question'].tolist()
answers = df_b['answer'].tolist()
# Инициализация Sentence Transformers
model = SentenceTransformer('all-MiniLM-L6-v2')
# Генерация эмбеддингов для контекстов
context_vectors = []
for context in tqdm(contexts):
    context_vector = model.encode(context, show_progress_bar=False)
    context_vectors.append(context_vector)
context_vectors = np.array(context_vectors, dtype='float32')
# Генерация эмбеддингов для вопросов
question_vectors = []
for question in tqdm(questions):
    question_vector = model.encode(question, show_progress_bar=False)
    question_vectors.append(question_vector)
question_vectors = np.array(question_vectors, dtype='float32')
# Нормализация векторов
context_vectors = normalize(context_vectors, norm='l2')
question_vectors = normalize(question_vectors, norm='l2')
# Проверка точности
def evaluate_embeddings(df_b, question_vectors, context_vectors):
    correct = 0
    for i in tqdm(range(len(df_b)), unit="question"):
        true_answer = df_b.iloc[i]['answer']
        question_vector = question_vectors[i].reshape(1, -1)
        similarities = cosine_similarity(question_vector, context_vectors).flatten()
        top_index = np.argmax(similarities)
        retrieved_answer = df_b.iloc[top_index]['answer']
        if retrieved_answer == true_answer:
            correct += 1
    accuracy = correct / len(df_b)
    return accuracy
# Вычисление точности
accuracy_embeddings = evaluate_embeddings(df_b, question_vectors, context_vectors)
print(f'Sentence Transformers accuracy: {accuracy_embeddings * 100:.2f}%')

100%|██████████████████████████████████████████████████████████████████████| 11983/11983 [04:30<00:00, 44.36question/s]

Sentence Transformers accuracy: 75.52%


In [23]:
# FAISS with Sentence Transformers
# Инициализация Sentence Transformers
model = SentenceTransformer('all-MiniLM-L6-v2')
# Генерация эмбеддингов для контекстов
context_vectors = []
for context in tqdm(contexts):
    context_vector = model.encode(context, show_progress_bar=False)
    context_vectors.append(context_vector)
context_vectors = np.array(context_vectors, dtype='float32')
# Генерация эмбеддингов для вопросов
question_vectors = []
for question in tqdm(questions):
    question_vector = model.encode(question, show_progress_bar=False)
    question_vectors.append(question_vector)
question_vectors = np.array(question_vectors, dtype='float32')
# Нормализация векторов
context_vectors = normalize(context_vectors, norm='l2')
question_vectors = normalize(question_vectors, norm='l2')
# Создание индекса FAISS
dimension = context_vectors.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(context_vectors)
# Проверка точности FAISS с Sentence Transformers
def evaluate_faiss_embeddings(df_b, question_vectors, index):
    correct = 0
    for i in tqdm(range(len(df_b)), unit="question"):
        true_answer = df_b.iloc[i]['answer']
        question_vector = question_vectors[i].reshape(1, -1)
        distances, indices = index.search(question_vector, k=1)
        top_index = indices[0][0]
        retrieved_answer = df_b.iloc[top_index]['answer']
        if retrieved_answer == true_answer:
            correct += 1
    accuracy = correct / len(df_b)
    return accuracy
# Вычисление точности
accuracy_embeddings = evaluate_faiss_embeddings(df_b, question_vectors, index)
print(f'FAISS with Sentence Transformers accuracy: {accuracy_embeddings * 100:.2f}%')

100%|████████████████████████████████████████████████████████████████████| 11983/11983 [00:10<00:00, 1124.44question/s]

FAISS with Sentence Transformers accuracy: 75.52%


In [24]:
# Hybrid Search (BM25 + Sentence Transformers)
# Подготовка данных
contexts = df_b['context'].tolist()
questions = df_b['question'].tolist()
answers = df_b['answer'].tolist()
# Инициализация Sentence Transformers
model = SentenceTransformer('all-MiniLM-L6-v2')
# Инициализация BM25
tokenized_contexts = [context.split() for context in contexts]
bm25 = BM25Okapi(tokenized_contexts)
# Генерация эмбеддингов для контекстов
context_vectors = []
for context in tqdm(contexts):
    context_vector = model.encode(context, show_progress_bar=False)
    context_vectors.append(context_vector)
context_vectors = np.array(context_vectors, dtype='float32')
# Генерация эмбеддингов для вопросов
question_vectors = []
for question in tqdm(questions):
    question_vector = model.encode(question, show_progress_bar=False)
    question_vectors.append(question_vector)
question_vectors = np.array(question_vectors, dtype='float32')
# Гибридный поиск
def evaluate_hybrid_search(df_b, question_vectors, context_vectors, bm25, top_k=3):
    correct = 0
    for i in tqdm(range(len(df_b)), unit="question"):
        true_answer = df_b.iloc[i]['answer']
        question = df_b.iloc[i]['question']
        bm25_scores = bm25.get_scores(question.split())
        top_bm25_indices = np.argsort(bm25_scores)[::-1][:top_k]
        filtered_context_vectors = context_vectors[top_bm25_indices]
        question_vector = question_vectors[i].reshape(1, -1)
        similarities = cosine_similarity(question_vector, filtered_context_vectors).flatten()
        best_match_index = top_bm25_indices[np.argmax(similarities)]
        retrieved_answer = df_b.iloc[best_match_index]['answer']
        if retrieved_answer == true_answer:
            correct += 1
    accuracy = correct / len(df_b)
    return accuracy
# Вычисление точности
accuracy_hybrid = evaluate_hybrid_search(df_b, question_vectors, context_vectors, bm25, top_k=10)
print(f'Hybrid Search (BM25 + Sentence Transformers) accuracy: {accuracy_hybrid * 100:.2f}%')

100%|██████████████████████████████████████████████████████████████████████| 11983/11983 [12:04<00:00, 16.55question/s]

Hybrid Search (BM25 + Sentence Transformers) accuracy: 81.73%


In [25]:
# Hybrid Search (BM25 + Sentence Transformers) Проверка разных значенний top_k
# Подготовка данных
contexts = df_b['context'].tolist()
questions = df_b['question'].tolist()
answers = df_b['answer'].tolist()
# Инициализация Sentence Transformers
model = SentenceTransformer('all-MiniLM-L6-v2')
# Инициализация BM25
tokenized_contexts = [context.split() for context in contexts]
bm25 = BM25Okapi(tokenized_contexts)
# Генерация эмбеддингов для контекстов
context_vectors = []
for context in tqdm(contexts):
    context_vector = model.encode(context, show_progress_bar=False)
    context_vectors.append(context_vector)
context_vectors = np.array(context_vectors, dtype='float32')
# Генерация эмбеддингов для вопросов
question_vectors = []
for question in tqdm(questions):
    question_vector = model.encode(question, show_progress_bar=False)
    question_vectors.append(question_vector)
question_vectors = np.array(question_vectors, dtype='float32')
# Гибридный поиск
def evaluate_hybrid_search(df_b, question_vectors, context_vectors, bm25, top_k=5):
    correct = 0
    for i in tqdm(range(len(df_b)), unit="question"):
        true_answer = df_b.iloc[i]['answer']
        question = df_b.iloc[i]['question']
        bm25_scores = bm25.get_scores(question.split())
        top_bm25_indices = np.argsort(bm25_scores)[::-1][:top_k]
        filtered_context_vectors = context_vectors[top_bm25_indices]
        question_vector = question_vectors[i].reshape(1, -1)
        similarities = cosine_similarity(question_vector, filtered_context_vectors).flatten()
        best_match_index = top_bm25_indices[np.argmax(similarities)]
        retrieved_answer = df_b.iloc[best_match_index]['answer']
        if retrieved_answer == true_answer:
            correct += 1
    accuracy = correct / len(df_b)
    return accuracy
# Подбор параметра top_k
def tune_top_k(df_b, question_vectors, context_vectors, bm25, top_k_values):
    results = []
    for top_k in top_k_values:
        print(f"top_k = {top_k}")
        accuracy = evaluate_hybrid_search(df_b, question_vectors, context_vectors, bm25, top_k=top_k)
        results.append({"top_k": top_k, "accuracy": accuracy})
        print(f"top_k = {top_k}, accuracy = {accuracy * 100:.2f}%")
    return results
# Диапазон значений top_k
top_k_values = [2, 3, 4, 5, 6, 7]
results = tune_top_k(df_b, question_vectors, context_vectors, bm25, top_k_values)

100%|███████████████████████████████████████████████████████████████████████████| 11983/11983 [01:37<00:00, 122.89it/s]


top_k = 2


100%|██████████████████████████████████████████████████████████████████████| 11983/11983 [13:19<00:00, 14.99question/s]


top_k = 2, accuracy = 85.05%
top_k = 3


100%|██████████████████████████████████████████████████████████████████████| 11983/11983 [13:20<00:00, 14.98question/s]


...

top_k = 5, accuracy = 82.87%
top_k = 6


100%|██████████████████████████████████████████████████████████████████████| 11983/11983 [11:15<00:00, 17.73question/s]


top_k = 6, accuracy = 82.48%
top_k = 7


100%|██████████████████████████████████████████████████████████████████████| 11983/11983 [11:13<00:00, 17.79question/s]

top_k = 7, accuracy = 82.27%


In [207]:
# Hybrid Search (BM25 + Sentence Transformers) combine scores
# Подготовка данных
contexts = df_b['context'].tolist()
questions = df_b['question'].tolist()
answers = df_b['answer'].tolist()
# Инициализация Sentence Transformers
model = SentenceTransformer('all-MiniLM-L6-v2')
# Инициализация BM25
tokenized_contexts = [context.split() for context in contexts]
bm25 = BM25Okapi(tokenized_contexts)
# Генерация эмбеддингов для контекстов
context_vectors = []
for context in tqdm(contexts):
    context_vector = model.encode(context, show_progress_bar=False)
    context_vectors.append(context_vector)
context_vectors = np.array(context_vectors, dtype='float32')
# Генерация эмбеддингов для вопросов
question_vectors = []
for question in tqdm(questions):
    question_vector = model.encode(question, show_progress_bar=False)
    question_vectors.append(question_vector)
question_vectors = np.array(question_vectors, dtype='float32')
# Гибридный поиск
def evaluate_hybrid_search(df_b, question_vectors, context_vectors, bm25, top_k=3):
    correct = 0
    for i in tqdm(range(len(df_b)), unit="question"):
        true_answer = df_b.iloc[i]['answer']
        question = df_b.iloc[i]['question']
        bm25_scores = bm25.get_scores(question.split())
        top_bm25_indices = np.argsort(bm25_scores)[::-1][:top_k]
        filtered_context_vectors = context_vectors[top_bm25_indices]
        question_vector = question_vectors[i].reshape(1, -1)
        similarities = cosine_similarity(question_vector, filtered_context_vectors).flatten()
# Комбинирование оценок BM25 и косинусного сходства
        combined_scores = bm25_scores[top_bm25_indices] + similarities
        best_match_index = top_bm25_indices[np.argmax(combined_scores)]
        retrieved_answer = df_b.iloc[best_match_index]['answer']
        if retrieved_answer == true_answer:
            correct += 1
    accuracy = correct / len(df_b)
    return accuracy
# Вычисление точности
accuracy_hybrid = evaluate_hybrid_search(df_b, question_vectors, context_vectors, bm25, top_k=10)
print(f'Hybrid Search (BM25 + Sentence Transformers) combine scores accuracy: {accuracy_hybrid * 100:.2f}%')

100%|██████████████████████████████████████████████████████████████████████| 11983/11983 [30:19<00:00,  6.59question/s]

Hybrid Search (BM25 + Sentence Transformers) combine scores accuracy: 87.60%


In [206]:
#RRF
# Подготовка данных
contexts = df_b['context'].tolist()
questions = df_b['question'].tolist()
answers = df_b['answer'].tolist()
# Инициализация Sentence Transformers
model = SentenceTransformer('all-MiniLM-L6-v2')
# Инициализация BM25
tokenized_contexts = [context.split() for context in contexts]
bm25 = BM25Okapi(tokenized_contexts)
# Генерация эмбеддингов для контекстов
context_vectors = []
for context in tqdm(contexts):
    context_vector = model.encode(context, show_progress_bar=False)
    context_vectors.append(context_vector)
context_vectors = np.array(context_vectors, dtype='float32')
# Генерация эмбеддингов для вопросов
question_vectors = []
for question in tqdm(questions):
    question_vector = model.encode(question, show_progress_bar=False)
    question_vectors.append(question_vector)
question_vectors = np.array(question_vectors, dtype='float32')
# Гибридный поиск с использованием RRF
def evaluate_rrf(df_b, question_vectors, context_vectors, bm25, k=60):
    correct = 0
    for i in tqdm(range(len(df_b)), unit="question"):
        true_answer = df_b.iloc[i]['answer']
        question = df_b.iloc[i]['question']
        bm25_scores = bm25.get_scores(question.split())
        bm25_ranks = np.argsort(bm25_scores)[::-1]
        question_vector = question_vectors[i].reshape(1, -1)
        similarities = cosine_similarity(question_vector, context_vectors).flatten()
        st_ranks = np.argsort(similarities)[::-1]
        # Комбинирование рангов с использованием RRF
        rrf_scores = {idx: 0 for idx in range(len(df_b))}
        for rank, idx in enumerate(bm25_ranks[:k]):
            rrf_scores[idx] += 1 / (60 + rank)
        for rank, idx in enumerate(st_ranks[:k]):
            rrf_scores[idx] += 1 / (60 + rank)
        best_match_index = max(rrf_scores, key=rrf_scores.get)
        retrieved_answer = df_b.iloc[best_match_index]['answer']
        if retrieved_answer == true_answer:
            correct += 1
    accuracy = correct / len(df_b)
    return accuracy
# Вычисление точности
accuracy_rrf = evaluate_rrf(df_b, question_vectors, context_vectors, bm25, k=60)
print(f'RRF accuracy: {accuracy_rrf * 100:.2f}%')

100%|██████████████████████████████████████████████████████████████████████| 11983/11983 [55:14<00:00,  3.62question/s]

RRF accuracy: 83.25%


Использование гибридного поиска с отбором кандидатов только ухудшило результат, при увеличении отобранных элементов с помощью BM25 показатель accuracy снижается.

Метод гибридного поиска из комбинации BM25 и Sentence Transformers с помощью линейного сложения их оценок показал наилучшие результаты accuracy: 87.60%

Дополнительно интересно отметить что использование FAISS с Sentence Transformers ускорило поиск в 43 раза, но так же стоит отметить, что использовались разные метрики: для FAISS евклидово расстояние, для Sentence Transformers косинусное расстояние. Следует произвести дополнительные исследования

# Cоздание коллекции qdrant

In [145]:
data = [
    {"_id": idx, "context": row["context"], "question": row["question"], "answer": row["answer"]}
    for idx, row in df_b.iterrows()
]
data[:3]

[{'_id': 0,
  'context': 'caption tasmanian berry grower nic hansen showing macau chef antimo merone around his property as part of export engagement activities the rise and rise of the australian strawberry raspberry and blackberry industries has seen the sectors redouble their international trade focus with the release of a dedicated export plan to grow their global presence over the next 10 years driven by significant grower input the berry export summary 2028 maps the sectors current position where they want to be highopportunity markets and next steps hort innovation trade manager jenny van de meeberg said the value and volume of raspberry and blackberry exports rose by 100 per cent between 2016 and 2017 she said the australian strawberry industry experienced similar success with an almost 30 per cent rise in export volume and a 26 per cent rise in value to 326m over the same period australian berry sectors are in a firm position at the moment she said production adoption of prote

In [146]:
type(data)

list

In [152]:
!pip install qdrant_client datasets fastembed

In [153]:
# Инициализация модели
bm25_embedding_model = SparseTextEmbedding("Qdrant/bm25")
# Извлечение текстов из "context"
contexts = [item["context"] for item in data]
# Создание эмбеддингов для всех текстов
bm25_embeddings = list(bm25_embedding_model.passage_embed(contexts))
# Проверка количества эмбеддингов
len(bm25_embeddings)

Fetching 29 files:   0%|          | 0/29 [00:00<?, ?it/s]

11983

In [154]:
# Инициализация модели
dense_embedding_model = TextEmbedding("sentence-transformers/all-MiniLM-L6-v2")
# Извлечение текста из "context"
texts = [item["context"] for item in data]
# Создание эмбеддингов для всех текстов с прогресс-баром
dense_embeddings = list(
    tqdm(dense_embedding_model.passage_embed(texts), total=len(texts))
)
# Проверка количества эмбеддингов
len(dense_embeddings)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████| 11983/11983 [04:06<00:00, 48.67it/s]


11983

In [155]:
len(dense_embeddings[0])

384

In [156]:
# Инициализация модели
late_interaction_embedding_model = LateInteractionTextEmbedding("colbert-ir/colbertv2.0")
# Функция для разбиения текста на части
def chunk_text(text, max_length):
    return [text[i:i + max_length] for i in range(0, len(text), max_length)]
# Функция для обработки одного текста
def process_context(context, max_length, model):
    chunks = chunk_text(context, max_length)
    chunk_embeddings = [
        np.array(list(model.passage_embed([chunk]))[0])
        for chunk in chunks
    ]
    return np.concatenate(chunk_embeddings, axis=0)
# Извлечение текстов и обработка с многозадачностью
contexts = [item["context"] for item in data]
max_length = 1024
final_embeddings = []
with ThreadPoolExecutor() as executor:
    futures = [
        executor.submit(process_context, context, max_length, late_interaction_embedding_model)
        for context in contexts
    ]
    for future in tqdm(futures):
        final_embeddings.append(future.result())
len(final_embeddings)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████| 11983/11983 [1:15:30<00:00,  2.64it/s]


11983

In [157]:
len(final_embeddings[0])

650

In [ ]:
!docker run -d -p 6333:6333 -p 6334:6334 qdrant/qdrant:v1.10.0

In [184]:
client.delete_collection("RAG")

True

In [159]:
# Создание qdrant коллекции
client = QdrantClient("http://localhost:6333", timeout=600)
client.create_collection(
    "RAG",
    vectors_config={
        "all-MiniLM-L6-v2": models.VectorParams(
            size=len(context_vectors[0]),
            distance=models.Distance.COSINE,
        ),
        "colbertv2.0": models.VectorParams(
            size=len(final_embeddings[0][0]),
            distance=models.Distance.COSINE,
            multivector_config=models.MultiVectorConfig(
                comparator=models.MultiVectorComparator.MAX_SIM,
            )
        ),
    },
    sparse_vectors_config = {
        "bm25": models.SparseVectorParams(
            modifier=models.Modifier.IDF,
        )
    }
)

True

In [162]:
batch_size = 4
for batch_index in tqdm(range(0, len(data), batch_size)):
    batch = data[batch_index:batch_index + batch_size]
    # Формируем точки для загрузки
    points = [
        models.PointStruct(
            id=int(item["_id"]),
            vector={
                "all-MiniLM-L6-v2": context_vectors[i].tolist(),
                "bm25": bm25_embeddings[i].as_object(),
                "colbertv2.0": final_embeddings[i].tolist(),
            },
            payload={
                "_id": item["_id"],
                "context": item["context"],
                "question": item["question"],
                "answer": item["answer"],
            }
        )
        for i, item in enumerate(batch)
    ]
    # Загружаем точки в хранилище
    client.upload_points(
        collection_name = "RAG",
        points=points,
    )

100%|██████████████████████████████████████████████████████████████████████████████| 2996/2996 [38:00<00:00,  1.31it/s]


In [163]:
client.get_collection("RAG")

CollectionInfo(status=<CollectionStatus.YELLOW: 'yellow'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=None, indexed_vectors_count=3996, points_count=13315, segments_count=9, config=CollectionConfig(params=CollectionParams(vectors={'all-MiniLM-L6-v2': VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), 'colbertv2.0': VectorParams(size=128, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=MultiVectorConfig(comparator=<MultiVectorComparator.MAX_SIM: 'max_sim'>))}, shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors={'bm25': SparseVectorParams(index=None, modifier=<Modifier.IDF: 'idf'>)}), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0,

In [183]:
response = client.count(
    collection_name="RAG"
)
print(f"'RAG' count: {response.count}")

'RAG' count: 11983


Коллекция создана успешно, для дальнейшего сравнения и экспериментов в коллекцию поместил вектора обработанные all-MiniLM-L6-v2, bm25, colbertv2.0 и payload всего очищенного датасета

# Тестирование данных в коллекции (в разработке)

In [ ]:
client = QdrantClient(url='http://localhost:6333')

In [198]:
# Тестирование all-MiniLM-L6-v2
# Извлекаем вектора и ответы из коллекции
context_vectors = []
answers = []
offset=None
while True:
    points, next_page = client.scroll(
        collection_name='RAG',
        scroll_filter={},
        limit = 200,
        offset=offset,
        with_vectors=['all-MiniLM-L6-v2'],
        with_payload=True
    )
# Добавляем вектора и ответы в списки
    for point in points:
        vector = point.vector['all-MiniLM-L6-v2']
        context_vectors.append(vector)
        answer = point.payload['answer']
        answers.append(answer)
#Проверяем, есть ли следующая страница
    if next_page is None:
        break
    offset = next_page
# Преобразуем список векторов в numpy массив
context_vectors = np.array(context_vectors, dtype='float32')
# Проверка точности Sentence Transformers
def evaluate_embeddings(df_b, question_vectors, context_vectors, answers):
    correct = 0
    for i in tqdm(range(len(df_b)), unit="question"):
        true_answer = df_b.iloc[i]['answer']
        question_vector = question_vectors[i].reshape(1, -1)
        similarities = cosine_similarity(question_vector, context_vectors).flatten()
        top_index = np.argmax(similarities)
        retrieved_answer = answers[top_index]
        if retrieved_answer == true_answer:
            correct += 1
    accuracy = correct / len(df_b)
    return accuracy
# Вычисление точности
accuracy_embeddings = evaluate_embeddings(df_b, question_vectors, context_vectors, answers)
print(f'Sentence Transformers accuracy: {accuracy_embeddings * 100:.2f}%')

100%|██████████████████████████████████████████████████████████████████████| 11983/11983 [05:55<00:00, 33.67question/s]

Sentence Transformers accuracy: 0.03%


В тестировании all-MiniLM-L6-v2 точность крайне низкая, вероятно присутствует баг

При проверке выявлена неккоректная загрузка векторов, исправлено - векторы до загрузки и после выгрузки соответствуют

Проблема сохраняется, измерения не изменились, требуется дополнительное исследование


# Данные для проверки, (создан для выявления причины бага вычисления Sentence Transformers accuracy после загрузки векторов в коллекцию, на данный момент в разработке)

In [149]:
# Инициализация модели Sentence Transformers
model = SentenceTransformer('all-MiniLM-L6-v2')
# Создание эмбеддингов для вопросов
questions = df_b['question'].tolist()
answers = df_b['answer'].tolist()
question_embeddings = []
for question in tqdm(questions):
    question_embedding = model.encode(question, show_progress_bar=False)
    question_embeddings.append(question_embedding)
question_embeddings = np.array(question_embeddings, dtype='float32')
# Связь эмбеддингов с ответами в порядке, соответствующем оригиналу
question_embeddings_with_answers = [(embedding, answer) for embedding, answer in zip(question_embeddings, answers)]

100%|███████████████████████████████████████████████████████████████████████████| 11983/11983 [01:43<00:00, 116.18it/s]


In [150]:
# Нормализация векторов
question_embeddings = normalize(question_embeddings, norm='l2')
context_vectors = normalize(context_vectors, norm='l2')
# Проверка точности
def evaluate_embeddings(df_b, question_embeddings, context_vectors, answers):
    correct = 0
    for i in tqdm(range(len(df_b)), unit="question"):
        true_answer = df_b.iloc[i]['answer']
        question_embedding = question_embeddings[i].reshape(1, -1)
        distances = distance.cdist(question_embedding, context_vectors, metric='euclidean').flatten()
        top_index = np.argmin(distances)
        retrieved_answer = answers[top_index]
        if retrieved_answer == true_answer:
            correct += 1
    accuracy = correct / len(df_b)
    return accuracy
# Вычисление точности
accuracy_embeddings = evaluate_embeddings(df_b, question_embeddings, context_vectors, answers)
print(f'Sentence Transformers accuracy: {accuracy_embeddings * 100:.2f}%')

100%|██████████████████████████████████████████████████████████████████████| 11983/11983 [03:19<00:00, 60.19question/s]

Sentence Transformers accuracy: 0.03%


# Дополнительные эксперименты с коллекцией (в разработке)

In [13]:
!pip install ranx

In [17]:
client = QdrantClient("http://localhost:6333", timeout=600)
client.count("RAG")

UnexpectedResponse: Unexpected Response: 404 (Not Found)
Raw response content:
b'{"status":{"error":"Not found: Collection `RAG` doesn\'t exist!"},"time":7.687e-6}'

In [ ]:
collection_info = client.get_collection(collection_name="RAG")
collection_info

CollectionInfo(status=<CollectionStatus.YELLOW: 'yellow'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=None, indexed_vectors_count=35949, points_count=11983, segments_count=7, config=CollectionConfig(params=CollectionParams(vectors={'all-MiniLM-L6-v2': VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), 'colbertv2.0': VectorParams(size=128, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=MultiVectorConfig(comparator=<MultiVectorComparator.MAX_SIM: 'max_sim'>))}, shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors={'bm25': SparseVectorParams(index=None, modifier=<Modifier.IDF: 'idf'>)}), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0

In [ ]:
# Инициализация моделей эмбеддингов
dense_embedding_model = TextEmbedding("sentence-transformers/all-MiniLM-L6-v2")
bm25_embedding_model = SparseTextEmbedding("Qdrant/bm25")
late_interaction_embedding_model = LateInteractionTextEmbedding("colbert-ir/colbertv2.0")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Fetching 29 files:   0%|          | 0/29 [00:00<?, ?it/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
# Загрузка из коллекции RAG
queries, query_ids, answers, contexts = [], [], [], []
response = client.scroll(collection_name="RAG", scroll_filter=None, limit=12000)
for point in response[0]:
    queries.append(point.payload["question"])
    query_ids.append(point.payload["_id"])
    answers.append(point.payload["answer"])
    contexts.append(point.payload["context"])

In [ ]:
data = {
    "_id": query_ids,
    "question": queries,
    "answer": answers,
}
df_t = pd.DataFrame(data)

In [ ]:
df_t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11983 entries, 0 to 11982
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   _id       11983 non-null  int64 
 1   question  11983 non-null  object
 2   answer    11983 non-null  object
dtypes: int64(1), object(2)
memory usage: 281.0+ KB


In [ ]:
tqdm.pandas()
dense_embeddings = []
bm25_embeddings = []
late_interaction_embeddings = []
# Генерация эмбеддингов и их сохранение
for question in tqdm(df_t['question']):
    # Обработка dense_embedding
    dense_embedding = list(dense_embedding_model.embed(question))
    dense_embeddings.append(dense_embedding)
    # Обработка bm25_embedding
    bm25_embedding = list(bm25_embedding_model.embed(question))
    bm25_embeddings.append(bm25_embedding)
    # Обработка late_interaction_embedding
    late_interaction_embedding = list(late_interaction_embedding_model.embed(question))
    late_interaction_embeddings.append(late_interaction_embedding)
# Добавляем эмбеддинги в датафрейм
df_t['dense_embedding'] = dense_embeddings
df_t['bm25_embedding'] = bm25_embeddings
df_t['late_interaction_embedding'] = late_interaction_embeddings

100%|████████████████████████████████████████████████████████████████████████████| 11983/11983 [21:07<00:00,  9.45it/s]


In [ ]:
# Создаем словари для хранения векторов с индексами равными значению из df_t['_id']
dense_vectors = {}
sparse_vectors = {}
late_vectors = {}
# Генерация эмбеддингов и сохранение их с индексами
for _id, query in tqdm(zip(df_t['_id'], df_t['question']), total=len(df_t)):
    # Сохранение эмбеддингов
    dense_vectors[_id] = next(dense_embedding_model.query_embed(query))
    sparse_vectors[_id] = next(bm25_embedding_model.query_embed(query))
    late_vectors[_id] = next(late_interaction_embedding_model.query_embed(query))
# Преобразуем словари обратно в DataFrame
df_t['dense_embedding'] = df_t['_id'].map(dense_vectors)
df_t['bm25_embedding'] = df_t['_id'].map(sparse_vectors)
df_t['late_interaction_embedding'] = df_t['_id'].map(late_vectors)
print(df_t.head())

Генерация эмбеддингов: 100%|█████████████████████████████████████████████████████| 11983/11983 [35:54<00:00,  5.56it/s]

   _id                                           question  \
0    0  what is the berry export summary 2028 and what...   
1    1  what are some of the benefits reported from ha...   
2    2  what are the unique features of the coolands f...   
3    3  what is the main difference between the nation...   
4    4  how did gunnar nelson win the fight against za...   

                                              answer  \
0  the berry export summary 2028 is a dedicated e...   
1  benefits reported from having access to selfsu...   
2  the unique features of the coolands for twitte...   
3  the main difference between the nss and the ih...   
4  gunnar nelson won the fight against zak cummin...   

                                     dense_embedding  \
0  [-0.08194435, -0.013052515, -0.044808928, 0.01...   
1  [-0.014962784, 0.057150137, 0.05230261, 0.0215...   
2  [-0.10045761, -0.022892212, 0.016677994, -0.03...   
3  [-0.022077423, -0.01830665, -0.03164753, -0.00...   
4  [-0.10147287,

In [ ]:
df_t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11983 entries, 0 to 11982
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   _id                         11983 non-null  int64 
 1   question                    11983 non-null  object
 2   answer                      11983 non-null  object
 3   dense_embedding             11983 non-null  object
 4   bm25_embedding              11983 non-null  object
 5   late_interaction_embedding  11983 non-null  object
dtypes: int64(1), object(5)
memory usage: 561.8+ KB
